<a href="https://colab.research.google.com/github/Salma-Jamal/Movie-Recommendation-System/blob/main/models/Movie_Recommendation_System_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.metrics import pairwise_distances
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import mean_absolute_error
import numpy as np

In [2]:
data = pd.read_csv("MovieData.csv")

In [3]:
data.head()

,Unnamed: 0,userId,movieId,rating,timestamp,title,genres
0,1,1,122,5.0,838985046,Boomerang (1992),Comedy|Romance
1,2,1,185,5.0,838983525,"Net, The (1995)",Action|Crime|Thriller
2,3,1,231,5.0,838983392,Dumb & Dumber (1994),Comedy
3,4,1,292,5.0,838983421,Outbreak (1995),Action|Drama|Sci-Fi|Thriller
4,5,1,316,5.0,838983392,Stargate (1994),Action|Adventure|Sci-Fi


In [4]:
# Create a user-item matrix
matrix = data.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)


In [5]:
matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,65006,65011,65025,65027,65037,65088,65091,65126,65130,65133
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71563,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71564,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71565,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# Calculate the pairwise distances between all items
item_dist = pairwise_distances(matrix.T, metric='cosine')

In [7]:
# Fit the NearestNeighbors model
knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=20, n_jobs=-1)
knn.fit(item_dist)

NearestNeighbors(algorithm='brute', metric='cosine', n_jobs=-1, n_neighbors=20)

In [8]:
# Use the fitted model to get the k nearest neighbors for each item
neighbors = knn.kneighbors(return_distance=False)

In [9]:
def RecommendMovies(UserID , neighborsMatrix):
  bestRatedMovie = data[data['userId'] == UserID].loc[:,['movieId',	'rating']].sort_values(by=['rating'])['movieId'].tolist()[-1]
  Nearest_movies = neighborsMatrix[bestRatedMovie]

  for movie_id in Nearest_movies[:10]:
    print(data[data['movieId'] == movie_id ]['title'].tolist()[0])

In [10]:
RecommendMovies(5,neighbors)

Postman, The (Postino, Il) (1994)
Fled (1996)
Floating (1997)
Santa Claus: The Movie (1985)
Romper Stomper (1992)
Batman (1989)
Cronos (1993)
Love Stinks (1999)
Hellraiser: Bloodline (1996)
Absolute Power (1997)


In [11]:
#Use the k nearest neighbors to predict the ratings for each user-item combination
predictions = matrix.dot(item_dist) / np.array([np.abs(item_dist).sum(axis=1)])

In [12]:
# Evaluate the accuracy of the predictions
mae = mean_absolute_error(predictions, matrix)
print('MAE:', mae)

MAE: 0.0894574829448961
